In [1]:
#Math/Data manipulation
import numpy as np
import pandas as pd

#Plotting
import matplotlib.pyplot as plt
import seaborn as sns

#ML packages
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

In [6]:
path_prefix = './test_GRIP_31may2021/GRIP_raw/'

In [7]:
data_overview = pd.read_csv(path_prefix+'GRIP_3136_raw.csv') 
data_overview.head()

,Sample,Particle ID,Area (ABD),Area (Filled),Aspect Ratio,Biovolume (Cylinder),Biovolume (P. Spheroid),Biovolume (Sphere),Calibration Factor,Calibration Image,...,Sphere Volume,Sum Intensity,Symmetry,Time,Timestamp,Transparency,Volume (ABD),Volume (ESD),Width,imgpaths
0,0,1,9.85,9.85,0.61,50.00,16.19,23.27,0.2752,1,...,0.00,19521.0,0.80,15:56:41,2020-11-20T14:56:41.039,0.10,23.270120,32.063020,2.75,/home/nico/Desktop/MarieCurie/Flowcam/test/GRI...
1,1,2,0.68,0.68,0.84,1.77,0.38,0.42,0.2752,1,...,0.00,3918.0,0.00,15:56:41,2020-11-20T14:56:41.313,0.04,0.423324,0.481389,0.83,/home/nico/Desktop/MarieCurie/Flowcam/test/GRI...
2,2,3,10.19,10.19,0.68,46.44,18.72,24.48,0.2752,1,...,0.00,23634.0,0.83,15:56:41,2020-11-20T14:56:41.583,0.07,24.475530,30.728550,2.89,/home/nico/Desktop/MarieCurie/Flowcam/test/GRI...
3,3,4,178.99,178.99,0.57,1980.59,1342.22,1801.36,0.2752,1,...,4.98,169831.0,0.79,15:56:41,2020-11-20T14:56:41.766,0.11,1801.358000,2522.085000,11.31,/home/nico/Desktop/MarieCurie/Flowcam/test/GRI...
4,4,5,0.74,0.74,0.82,2.28,0.44,0.48,0.2752,1,...,9.76,3903.0,0.00,15:56:42,2020-11-20T14:56:42.039,0.06,0.477327,0.571251,0.92,/home/nico/Desktop/MarieCurie/Flowcam/test/GRI...


In [8]:
data_overview['imgpaths'][0]

'/home/nico/Desktop/MarieCurie/Flowcam/test/GRIP_raw/GRIP_3136_0_20_1/GRIP_3136_0_20_1_1.png'

In [4]:
def load_and_process_images(df):
  img_list = []
  target_size = 256
  for path in df['imgpaths']:
    img = load_img(path, color_mode='grayscale')
    largest_dim = np.argmax([img.size[0], img.size[1]])
    ratio = target_size / img.size[largest_dim]
    img = img.resize((int(img.size[0]*ratio), int(img.size[1]*ratio)))
    img = img_to_array(img)
    img *= (1./255)
    img = 1 - img
    img = np.pad(img[:,:,0], [( (target_size-img.shape[0]) // 2 , (target_size-img.shape[0]) // 2 + ((target_size - img.shape[0])%2)), ( (target_size-img.shape[1]) // 2 , (target_size-img.shape[1]) // 2 + ((target_size - img.shape[1])%2))], mode='constant', constant_values=img.min())
    img = img.reshape(256,256,1)
#    img = list(img)
    img_list.append(img)
  df['image'] = img_list
  return df

In [5]:
subset = pd.DataFrame()

subset['imgpaths'] = data_overview['imgpaths'].str.split('train/')
subset['imgpaths'] = path_prefix + subset['imgpaths'].str[1]

#subset = load_and_process_images(subset)

subset[['Area (ABD)', 'Area (Filled)', 'Aspect Ratio', 'Biovolume (Cylinder)', 'Biovolume (P. Spheroid)', 'Biovolume (Sphere)', 'Calibration Factor', 'Circle Fit', 'Circularity', 'Circularity (Hu)', 'Compactness', 'Convex Perimeter', 'Convexity', 'Diameter (ABD)', 'Diameter (ESD)', 'Edge Gradient', 'Elongation', 'Feret Angle Max', 'Feret Angle Min', 'Fiber Curl', 'Fiber Straightness', 'Filter Score', 'Geodesic Aspect Ratio', 'Geodesic Length', 'Geodesic Thickness', 'Intensity', 'Length', 'Particles Per Chain', 'Perimeter', 'Roughness', 'Sigma Intensity', 'Sphere Complement', 'Sphere Count', 'Sphere Unknown', 'Sphere Volume', 'Sum Intensity', 'Symmetry', 'Transparency', 'Volume (ABD)', 'Volume (ESD)', 'Width']] = data_overview[['Area (ABD)', 'Area (Filled)', 'Aspect Ratio', 'Biovolume (Cylinder)', 'Biovolume (P. Spheroid)', 'Biovolume (Sphere)', 'Calibration Factor', 'Circle Fit', 'Circularity', 'Circularity (Hu)', 'Compactness', 'Convex Perimeter', 'Convexity', 'Diameter (ABD)', 'Diameter (ESD)', 'Edge Gradient', 'Elongation', 'Feret Angle Max', 'Feret Angle Min', 'Fiber Curl', 'Fiber Straightness', 'Filter Score', 'Geodesic Aspect Ratio', 'Geodesic Length', 'Geodesic Thickness', 'Intensity', 'Length', 'Particles Per Chain', 'Perimeter', 'Roughness', 'Sigma Intensity', 'Sphere Complement', 'Sphere Count', 'Sphere Unknown', 'Sphere Volume', 'Sum Intensity', 'Symmetry', 'Transparency', 'Volume (ABD)', 'Volume (ESD)', 'Width']]

subset['Label'] = None
subset['Label'][data_overview['camp']==1] = 0
subset['Label'][data_overview['corylus']==1] = 1
subset['Label'][data_overview['dust']==1] = 2
subset['Label'][data_overview['grim']==1] = 3
subset['Label'][data_overview['qrob']==1] = 4
subset['Label'][data_overview['qsub']==1] = 5
subset.head()

<ipython-input-5-f04bf1d28d21>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['Label'][data_overview['camp']==1] = 0
<ipython-input-5-f04bf1d28d21>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['Label'][data_overview['corylus']==1] = 1
<ipython-input-5-f04bf1d28d21>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['Label'][data_overview['dust']==1] = 2
<ipython-input-5-f04bf1d28d21>:14: SettingWithCopyWarni

,imgpaths,Area (ABD),Area (Filled),Aspect Ratio,Biovolume (Cylinder),Biovolume (P. Spheroid),Biovolume (Sphere),Calibration Factor,Circle Fit,Circularity,...,Sphere Count,Sphere Unknown,Sphere Volume,Sum Intensity,Symmetry,Transparency,Volume (ABD),Volume (ESD),Width,Label
0,train/campanian/camp_11nov_4_fc80_20x_1.png,41.72,41.72,0.70,283.50,164.42,202.72,0.2752,0.74,0.88,...,0,0.0,0.00,42841.0,0.90,0.05,202.72440,238.8423,6.12,0
1,train/campanian/camp_11nov_4_fc80_20x_2.png,22.96,22.96,0.62,133.54,60.80,82.78,0.2752,0.67,0.89,...,0,0.0,0.00,44052.0,0.87,0.08,82.77962,105.6288,4.04,0
2,train/campanian/camp_11nov_4_fc80_20x_3.png,396.76,402.93,0.36,3255.70,3685.72,5945.06,0.2752,0.02,0.42,...,3,0.0,191.59,652610.0,0.68,0.21,5945.05700,12171.9000,14.29,0
3,train/campanian/camp_11nov_4_fc80_20x_4.png,41.14,42.30,0.64,263.21,161.45,198.50,0.2752,0.63,0.73,...,0,0.0,0.00,57704.0,0.73,0.12,198.49500,292.0631,5.91,0
4,train/campanian/camp_11nov_4_fc80_20x_5.png,23.83,23.83,0.64,143.20,65.85,87.50,0.2752,0.69,0.87,...,0,0.0,0.00,47112.0,0.81,0.08,87.49621,111.3132,4.28,0


In [6]:
#plt.imshow(subset['image'][4])

In [7]:
#subset.to_pickle('./preprocessedData.zip', protocol=4)
#subset.to_csv('./CleanedData.csv', index=False)
subset.to_pickle('./CleanedData.pkl')
#subset.to_pickle('./preprocessedData.pkl.gz', compression='gzip', protocol=4)

In [8]:
#df_data = pd.read_csv('./CleanedData.csv')
#df_data = pd.read_pickle('./preprocessedData_wPadding.pkl')
#df_data = pd.read_pickle('./preprocessedData.pkl.gz', compression='gzip')

In [9]:
#plt.imshow(df_data['image'][4][:,:,0])

In [10]:
#df_data.head()

In [11]:
#X = df_data['image']
#y = df_data['Label']

In [12]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
df_data = pd.read_pickle('./CleanedData.pkl')

In [16]:
df_data['imgpaths'][:5].to_numpy()

array(['train/campanian/camp_11nov_4_fc80_20x_1.png',
       'train/campanian/camp_11nov_4_fc80_20x_2.png',
       'train/campanian/camp_11nov_4_fc80_20x_3.png',
       'train/campanian/camp_11nov_4_fc80_20x_4.png',
       'train/campanian/camp_11nov_4_fc80_20x_5.png'], dtype=object)

In [17]:
list_ds = tf.data.Dataset.list_files(df_data['imgpaths'].to_numpy(), shuffle=False)

In [18]:
for f in list_ds.take(5):
  print(f.numpy())

b'train\\campanian\\camp_11nov_10_fc80_20x_1.png'
b'train\\campanian\\camp_11nov_10_fc80_20x_10.png'
b'train\\campanian\\camp_11nov_10_fc80_20x_100.png'
b'train\\campanian\\camp_11nov_10_fc80_20x_101.png'
b'train\\campanian\\camp_11nov_10_fc80_20x_102.png'
